<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 3*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Using your baseline model from yesterday, hyperparameter tune it and report on your highest validation accuracy. Your singular goal today is to achieve the highest accuracy possible.

*Don't forgot to switch to GPU on Colab!*

### Hyperparameters to Tune

At a minimum, tune each of these hyperparameters using any strategy we discussed during lecture today: 
- Optimizer
- Learning Rate
- Activiation Function
  - At least 1 subparameter within the Relu activation function
- Number of Neurons in Hidden Layers
- Number of Hidden Layers
- Weight Initialization

In [30]:
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, Ftrl, Nadam
from sklearn.model_selection import train_test_split
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
import tensorflow as tf
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
import os

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [31]:
data = np.load('../quickdraw10.npz')
data.files
    
def load_quickdraw10(path, x_loc, y_loc, outlier_val=-1, test_size=0.4):
    """
    Get loaded data and split; returns X_train/test and y train_test
    with 40% split.
    Normalize data; enter in outlier value or keep default (-1) to 
    select auto and find X max.
    
    Input: data path, X and y index label
    
    :Return: X_train, X_test, y_train, y_test
    """
    X = path[x_loc]
    y = path[y_loc]
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        y, 
                                                        test_size = test_size,
                                                        random_state=445642)
    # Read or find outlier, -1 is auto, else inputed value used.
    if outlier_val == -1:
        outlier_val = X.max()
    else:
        outlier_val = outlier_val
    
    # Normalize X_test/train by outlier value
    X_train, X_test = X_train.astype('float32') / outlier_val, X_test.astype('float32') / outlier_val
    
    # Print info
    print(f'X shape = {X.shape}\n'
          f'y shape = {y.shape}\n'
          f'X (max, min) = ({X.max()}, {X.min()})\n'
          f'X (train.shape, test.shape) = [ {X_train.shape}, {X_test.shape} ]\n\n'
          f'y Unique = {len(np.unique(y))}')
    
    return X_train, y_train, X_test, y_test

# Set test and train sets
X_train, y_train, X_test, y_test = load_quickdraw10(path=data,
                                                   x_loc='arr_0',
                                                   y_loc='arr_1')

X shape = (100000, 784)
y shape = (100000,)
X (max, min) = (255, 0)
X (train.shape, test.shape) = [ (60000, 784), (40000, 784) ]

y Unique = 10


In [34]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16,32,100,200,300]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(0.001,0.1))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
METRIC_ACCURACY = 'accuracy'
GRID_SEARCH_RESULTS_DIR = 'logs/hparam_tuning'

In [35]:
# creating a dir to save/log our gridsearch results for use with tensorboard 
with tf.summary.create_file_writer(GRID_SEARCH_RESULTS_DIR).as_default():
    hp.hparams_config(
        # store h-params and their values 
        hparams=[HP_NUM_UNITS, HP_LEARNING_RATE, HP_OPTIMIZER],
        # store metrics to score the model 
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')])

In [44]:
def create_model(X_train, X_test, y_train, 
                 y_test, hparams, opt=None, 
                 epochs=10):
    """
    SGD, Adam, Adadelta, Ftrl, nadam or hparams
    """
    if opt == None:
        opt = hparams[HP_OPTIMIZER]
    elif opt.lower() == 'sgd':
        opt = SGD(learning_rate=hparams[HP_LEARNING_RATE])
    elif opt.lower() == 'adam':
        opt = Adam(learning_rate=hparams[HP_LEARNING_RATE])
    elif opt.lower() == 'adadelta':
        opt = Adadelta(learning_rate=hparams[HP_LEARNING_RATE])
    elif opt.lower() == 'ftrl':
        opt = Ftrl(learning_rate=hparams[HP_LEARNING_RATE])
    elif opt.lower() == 'nadam':
        opt = Nadam(learning_rate=hparams[HP_LEARNING_RATE])
    else:
        raise ValueError(f"Unknown optimizer input of {opt} used")
    
    
    # Model Creation
    model = Sequential([
        # Layer 1
        Dense(hparams[HP_NUM_UNITS],
              activation='relu'),
        # Output Layer
        Dense(10,
              activation='softmax')
    ])
    
    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(X_train, y_train, epochs=epochs)
    _, accuracy = model.evaluate(X_test, y_test)
    
    return accuracy

In [46]:
def run(run_dir, hparams):

    with tf.summary.create_file_writer(run_dir).as_default():
        # record the values used in this trial
        hp.hparams(hparams)  

        # call create_model to build, train, and score model on parameter values 
        accuracy = create_model(
                                hparams=hparams, 
                                X_train=X_train, X_test=X_test,
                                y_train=y_train, y_test=y_test,
                                opt=None, epochs=20
                               )

        # store trained accuracy to file 
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        
session_num = 0 

for num_units in HP_NUM_UNITS.domain.values:
    for learning_rate in (HP_LEARNING_RATE.domain.min_value, HP_LEARNING_RATE.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:

            # as we loop through all the hyper-param values
            # the store each unique combination in the dictionary hparams
            hparams = {
               HP_NUM_UNITS: num_units,
               HP_LEARNING_RATE: learning_rate,
               HP_OPTIMIZER: optimizer
            }

            run_name = f"run-{session_num}"
            print(f"--- Starting trial: {run_name}")
            print({h.name: hparams[h] for h in hparams})

            # exectue the run function, which runs the training of the models 
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'learning_rate': 0.001, 'optimizer': 'adam'}
Epoch 1/20
1875/1875 [==============================] - 1s 596us/step - loss: 1.1494 - accuracy: 0.6396
Epoch 2/20
1875/1875 [==============================] - 1s 589us/step - loss: 0.7493 - accuracy: 0.7774
Epoch 3/20
1875/1875 [==============================] - 1s 599us/step - loss: 0.6965 - accuracy: 0.7961
Epoch 4/20
1875/1875 [==============================] - 1s 576us/step - loss: 0.6727 - accuracy: 0.8024
Epoch 5/20
1875/1875 [==============================] - 1s 577us/step - loss: 0.6522 - accuracy: 0.8054
Epoch 6/20
1875/1875 [==============================] - 1s 595us/step - loss: 0.6318 - accuracy: 0.8122
Epoch 7/20
1875/1875 [==============================] - 1s 601us/step - loss: 0.6244 - accuracy: 0.8148
Epoch 8/20
1875/1875 [==============================] - 1s 593us/step - loss: 0.6187 - accuracy: 0.8154
Epoch 9/20
1875/1875 [==============================] - 1s 597us/step - loss: 0

1875/1875 [==============================] - 1s 512us/step - loss: 0.6764 - accuracy: 0.8074
Epoch 14/20
1875/1875 [==============================] - 1s 512us/step - loss: 0.6678 - accuracy: 0.8088
Epoch 15/20
1875/1875 [==============================] - 1s 487us/step - loss: 0.6676 - accuracy: 0.8099
Epoch 16/20
1875/1875 [==============================] - 1s 505us/step - loss: 0.6691 - accuracy: 0.8100
Epoch 17/20
1875/1875 [==============================] - 1s 508us/step - loss: 0.6555 - accuracy: 0.8107
Epoch 18/20
1875/1875 [==============================] - 1s 515us/step - loss: 0.6494 - accuracy: 0.8132
Epoch 19/20
1875/1875 [==============================] - 1s 506us/step - loss: 0.6459 - accuracy: 0.8159
Epoch 20/20
1250/1250 [==============================] - 1s 437us/step - loss: 0.6904 - accuracy: 0.8028
--- Starting trial: run-4
{'num_units': 32, 'learning_rate': 0.001, 'optimizer': 'adam'}
Epoch 1/20
1875/1875 [==============================] - 1s 600us/step - loss: 1.041

1875/1875 [==============================] - 1s 537us/step - loss: 0.7714 - accuracy: 0.7799
Epoch 5/20
1875/1875 [==============================] - 1s 532us/step - loss: 0.7389 - accuracy: 0.7855
Epoch 6/20
1875/1875 [==============================] - 1s 540us/step - loss: 0.7056 - accuracy: 0.7981
Epoch 7/20
1875/1875 [==============================] - 1s 535us/step - loss: 0.6847 - accuracy: 0.8041
Epoch 8/20
1875/1875 [==============================] - 1s 535us/step - loss: 0.6625 - accuracy: 0.8085
Epoch 9/20
1875/1875 [==============================] - 1s 535us/step - loss: 0.6512 - accuracy: 0.8113
Epoch 10/20
1875/1875 [==============================] - 1s 531us/step - loss: 0.6231 - accuracy: 0.8215
Epoch 11/20
1875/1875 [==============================] - 1s 536us/step - loss: 0.6141 - accuracy: 0.8216
Epoch 12/20
1875/1875 [==============================] - 1s 533us/step - loss: 0.6030 - accuracy: 0.8252
Epoch 13/20
1875/1875 [==============================] - 1s 534us/step -

1875/1875 [==============================] - 1s 626us/step - loss: 0.1674 - accuracy: 0.9506
Epoch 18/20
1875/1875 [==============================] - 1s 622us/step - loss: 0.1614 - accuracy: 0.9504
Epoch 19/20
1875/1875 [==============================] - 1s 622us/step - loss: 0.1479 - accuracy: 0.9566
Epoch 20/20
1250/1250 [==============================] - 1s 464us/step - loss: 0.7121 - accuracy: 0.8433
--- Starting trial: run-11
{'num_units': 100, 'learning_rate': 0.1, 'optimizer': 'sgd'}
Epoch 1/20
1875/1875 [==============================] - 1s 578us/step - loss: 1.4518 - accuracy: 0.5550
Epoch 2/20
1875/1875 [==============================] - 1s 660us/step - loss: 0.8759 - accuracy: 0.7496
Epoch 3/20
1875/1875 [==============================] - 1s 663us/step - loss: 0.7822 - accuracy: 0.7746
Epoch 4/20
1875/1875 [==============================] - 1s 565us/step - loss: 0.7308 - accuracy: 0.7926
Epoch 5/20
1875/1875 [==============================] - 1s 586us/step - loss: 0.6781 - a

1875/1875 [==============================] - 1s 703us/step - loss: 0.1953 - accuracy: 0.9426
Epoch 9/20
1875/1875 [==============================] - 1s 728us/step - loss: 0.1749 - accuracy: 0.9478
Epoch 10/20
1875/1875 [==============================] - 1s 707us/step - loss: 0.1431 - accuracy: 0.9568
Epoch 11/20
1875/1875 [==============================] - 2s 887us/step - loss: 0.1302 - accuracy: 0.9634
Epoch 12/20
1875/1875 [==============================] - 1s 748us/step - loss: 0.1128 - accuracy: 0.9678
Epoch 13/20
1875/1875 [==============================] - 1s 669us/step - loss: 0.0990 - accuracy: 0.9719
Epoch 14/20
1875/1875 [==============================] - 1s 675us/step - loss: 0.0851 - accuracy: 0.9752
Epoch 15/20
1875/1875 [==============================] - 2s 896us/step - loss: 0.0789 - accuracy: 0.9766
Epoch 16/20
1875/1875 [==============================] - 1s 740us/step - loss: 0.0725 - accuracy: 0.9789
Epoch 17/20
1875/1875 [==============================] - 1s 691us/st

1250/1250 [==============================] - 1s 507us/step - loss: 0.4900 - accuracy: 0.8587
--- Starting trial: run-18
{'num_units': 300, 'learning_rate': 0.1, 'optimizer': 'adam'}
Epoch 1/20
1875/1875 [==============================] - 2s 818us/step - loss: 0.8223 - accuracy: 0.7506
Epoch 2/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4613 - accuracy: 0.8630
Epoch 3/20
1875/1875 [==============================] - 2s 890us/step - loss: 0.3793 - accuracy: 0.8880
Epoch 4/20
1875/1875 [==============================] - 1s 730us/step - loss: 0.3069 - accuracy: 0.9080
Epoch 5/20
1875/1875 [==============================] - 1s 737us/step - loss: 0.2559 - accuracy: 0.9243
Epoch 6/20
1875/1875 [==============================] - 1s 720us/step - loss: 0.2136 - accuracy: 0.9346
Epoch 7/20
1875/1875 [==============================] - 1s 714us/step - loss: 0.1729 - accuracy: 0.9483
Epoch 8/20
1875/1875 [==============================] - 1s 721us/step - loss: 0.1487 - accur

In [48]:
%tensorboard --logdir logs/hparam_tuning

Reusing TensorBoard on port 6007 (pid 41679), started 0:25:11 ago. (Use '!kill 41679' to kill it.)

### Stretch Goals
- Implement Bayesian Hyper-parameter Optimization
- Select a new dataset and apply a neural network to it.
- Use a cloud base experiment tracking framework such as weights and biases
- Research potential architecture ideas for this problem. Try Lenet-10 for example. 